# Driver Overview

Let's run the WPS prep utility `ungrib` to transform a GFS 0-hour forecast GRIB files into initial conditions suitable for WRF or MPAS.

First install `ungrib` into our conda environment:

In [34]:
%%bash
env=$CONDA_PREFIX
eval "$($CONDA shell.bash hook)"
conda activate $env
conda install -y -c paul.madden ungrib >install-ungrib.out
conda deactivate && conda activate $env

`cd` into the `configs/` directory:

In [35]:
import os
os.chdir("../configs")

Here's the config we'll use:

In [36]:
!cat do00.yaml

data:
  '{{ gfs }}': https://noaa-gfs-bdp-pds.s3.amazonaws.com/gfs.{{ val.yyyymmdd }}/{{ val.hh }}/atmos/gfs.t{{ val.hh }}z.pgrb2.0p25.f000
ungrib:
  execution:
    executable: ungrib
  gribfiles:
    - '{{ val.gfs }}'
  rundir: '{{ val.rundir }}'
  start: '{{ cycle }}'
  step: 1
  stop: '{{ cycle }}'
  vtable: '{{ "WPSFILES" | env }}/Vtable.GFS'
val:
  gfs: '{{ val.rundir }}/anl.grib2'
  hh: '{{ cycle.strftime("%H") }}'
  rundir: '{{ "PWD" | env }}/driver-overview'
  yyyymmdd: '{{ cycle.strftime("%Y%m%d") }}'


First, use `uw fs` to stage the GFS GRIB file from AWS:

In [31]:
!uw fs copy -c do00.yaml --key-path data --cycle 2025-09-21T06

[2025-09-22T18:48:32]     INFO Validating config against internal schema: files-to-stage
[2025-09-22T18:48:32]     INFO Schema validation succeeded for fs config
[2025-09-22T18:48:32]    ERROR Relative path '{{ gfs }}' requires target directory to be specified


Now run `ungrib` via the UW driver:

In [33]:
!uw ungrib run -c do00.yaml --cycle 2025-09-21T06

[2025-09-22T18:50:35]     INFO Validating config against internal schema: ungrib
[2025-09-22T18:50:35]     INFO Schema validation succeeded for ungrib config
[2025-09-22T18:50:35]     INFO Validating config against internal schema: platform
[2025-09-22T18:50:35]     INFO Schema validation succeeded for platform config
[2025-09-22T18:50:35]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Executing
[2025-09-22T18:50:35]     INFO 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/GRIBFILE.AAA: Ready
[2025-09-22T18:50:35]  WARNING File {{ "WPSFILES" | env }}/Vtable.GFS: Not ready [external asset]
[2025-09-22T18:50:35]     INFO 20250921 06Z ungrib GRIB files: Ready
[2025-09-22T18:50:35]  WARNING 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Madden/uwtools_training/configs/driver-overview/Vtable: Not ready
[2025-09-22T18:50:35]  WARNING 20250921 06Z ungrib /scratch4/BMC/rtruc/Paul.Mad